In [1]:
# 如题，这个工作做的是基线工作
# 正常和房颤的二分类，使用6个12导联数据集中的1导联，进行multi-one的训练和测试，训练域内验证，网络选取为resnet18
# 没有使用任何泛化方法（包括GRL）
# 创建时间 2023年12月17日
# 这个工作的结果将作为基线。
# 其实问题不大，基线工作可以通过调参来降低分类水平至所提出的新方法以下。不过这是不得已之举，还是先老老实实看看效果。
# 以上。

In [2]:
# 非常适合消融实验：只蒸馏，只GRL
# 引入更多的小细节可以更好微调结果。

In [3]:
!gpustat

vclab-gpuserver-57          Fri Mar  8 21:35:36 2024  515.65.01
[0] NVIDIA GeForce RTX 3090 | 31'C,   0 % |  1124 / 24576 MB | yuzhenqin(813M)
[1] NVIDIA GeForce RTX 3090 | 70'C,  45 % |  7213 / 24576 MB | yangli(3773M) yangli(3129M)
[2] NVIDIA GeForce RTX 3090 | 30'C,   0 % |  2270 / 24576 MB | yuzhenqin(1959M)
[3] NVIDIA GeForce RTX 3090 | 28'C,   0 % |  4180 / 24576 MB | yuzhenqin(3869M)
[4] NVIDIA GeForce RTX 3090 | 66'C,  57 % |  9070 / 24576 MB | yangli(3533M) yangli(2609M) yangli(2617M)
[5] NVIDIA GeForce RTX 3090 | 56'C,  20 % |  3842 / 24576 MB | yangli(3531M)
[6] NVIDIA GeForce RTX 3090 | 58'C,  10 % | 18736 / 24576 MB | robotics(18425M)
[7] NVIDIA GeForce RTX 3090 | 66'C,  44 % | 18736 / 24576 MB | robotics(18425M)


In [4]:
# box1

# 库函数的调用
from models.DG_models import generate_model
from datasets import train_loader,val_loader,test_loader
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as pltac
import os
os.environ["CUDA_VISIBLE_DEVICES"] ='5'
device='cuda:0'
import wfdb
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from scipy import signal
from tqdm import tqdm
import math
import time
import torch
from torch import nn
from torchsummary import summary
import sklearn.metrics
import csv
from utils import *
import copy

warm_epoch=2
total_epoch=20
base_lr=2e-3
batch_size=128
max_lr=base_lr*batch_size/256
min_lr=max_lr/100
numleads=1
# domain generalization
DG_method='DG_GR'
# None #'DG_GR' #'MMD' # 'CORAL'  # 'IRM' # 'CausIRL_MMD' # 'CausIRL_CORAL' # 'cls_awr_ali' # 'DG_GR+IRM' 
# 'DG_GR_ensemble' # 'origin_add_fft_amp' # 'origin_add_fft_phase' # 'origin_add_fft_amp_phase' # 'origin_add_fft_amp_DGGR'
# 'DGGR_smooth' #'origin_add_fft_amp_DGGR_smooth'
add_recons=None #'fft','ksqi','psqi','ssqi','bass',None
#ensemble_recons=['fft','ksqi'] # ['fft','ksqi','psqi','ssqi','bass'] 
MMD_iter=4
MMD_gamma=1.
irm_lambda=1e2
GRL_loss_beta=0.05
irm_penalty_anneal_iters=10000000 # don't penalty #500
DGGR_smooth_eps=0.4
ali_weight=0.5
train_val_test_split=[0.8,0.2,1] # 1 for inter
smote=False
SAM_flag=False
base_optimizer=torch.optim.Adam
loss_fn=nn.CrossEntropyLoss()
# cnn_Ag
# xresnet18,xresnet34,xresnet50,xresnet101,xresnet152
# inception1d
# resnet18,resnet34,resnet50,resnet101,resnet152
# resnet1d_wang
# seresnet_big_18,seresnet_big_34,seresnet_big_50,seresnet_big_101,seresnet_big_152
# densenet1d

base_model='cnn_Ag'
exp_num=10
class_num=3

if DG_method in ['DGGR_smooth','origin_add_fft_amp_DGGR_smooth'] :
    param_flag='AF_2class_smote-'+str(smote)+'-SAM-'+str(SAM_flag)+'-DG_method-'+str(DG_method)+'-beta-'+str(int(GRL_loss_beta*100))+'-eps-'+str(int(DGGR_smooth_eps*100))
else:
    param_flag='AF_2class_smote-'+str(smote)+'-SAM-'+str(SAM_flag)+'-DG_method-'+str(DG_method)+'-beta-'+str(int(GRL_loss_beta*100))

print('ok')

# 10次实验取平均

T_auc_dict_list=[]
T_pre_dict_list=[]
T_rec_dict_list=[]
T_f1_dict_list=[]
T_f1_normal_dict_list=[]
T_f1_AF_dict_list=[]
T_f1_other_dict_list=[]
T_acc_dict_list=[]


R_auc_dict_list=[]
R_pre_dict_list=[]
R_rec_dict_list=[]
R_f1_dict_list=[]
R_f1_normal_dict_list=[]
R_f1_AF_dict_list=[]
R_f1_other_dict_list=[]
R_acc_dict_list=[]





start_t=time.time()


train_data_list=['C','A','L','N','B','H'] # T,A,L,N,H,B       
val_data_list_A=['A']
val_data_list_B=['B']
val_data_list_C=['C']
val_data_list_H=['H']
val_data_list_L=['L']
val_data_list_N=['N']

test_T_data_list=['T_3cls']
test_R_data_list=['R_3cls']
        
train_datasloader_instance=train_loader(dataname=train_data_list,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,smote=smote,maple=True if DG_method=='maple' else False,class_num=class_num)
train_datasloader=train_datasloader_instance.loader()
val_datasloader_A=val_loader(dataname=val_data_list_A,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()
val_datasloader_B=val_loader(dataname=val_data_list_B,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()
val_datasloader_C=val_loader(dataname=val_data_list_C,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()
val_datasloader_H=val_loader(dataname=val_data_list_H,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()
val_datasloader_L=val_loader(dataname=val_data_list_L,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()
val_datasloader_N=val_loader(dataname=val_data_list_N,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()
test_T_datasloader=test_loader(dataname=test_T_data_list,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()
test_R_datasloader=test_loader(dataname=test_R_data_list,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()








for exp_idx in range(exp_num):
    
    print('current_exp_idx=',exp_idx)
    
    print('total time cost='+str(int(time.time()-start_t))+"s")
    
    mdst_path='/home/yangli/AF_DG_OODG/save_model_states'
    mdst_name=str(int(time.time()))+'_ABCLNH_TR_jixian_zhijiaGRL_cnnAg_zong_f1.pth'

        
    recons_model=None
    model=generate_model(base_model=base_model,input_channels=numleads, num_classes=class_num,DG_method=DG_method,domain_classes=len(train_data_list)).to(device)
    optimizer=generate_optimizer(base_optimizer=base_optimizer,model_parameters=model.parameters(),SAM_flag=SAM_flag,weight_decay=1e-4)
    best_val_f1 = 0.0
    best_model_weights = None
        
    for epoch in range(total_epoch):
        print('_____________________________________')
        print(' ')
        time.sleep(0.1)
        train_one_epoch(epoch, warm_epoch, total_epoch, model,max_lr,min_lr, loss_fn, optimizer, \
                        base_optimizer, train_datasloader, device, \
                         DG_method=DG_method,MMD_iter=MMD_iter,MMD_gamma=MMD_gamma, \
                         irm_lambda=irm_lambda,irm_penalty_anneal_iters=irm_penalty_anneal_iters, \
                        ali_weight=ali_weight,SAM_operate=SAM_flag,recons_model=recons_model,add_recons=add_recons,  \
                       DGGR_smooth_eps=DGGR_smooth_eps,theta=0,GRL_loss_beta=GRL_loss_beta,class_num=class_num)

        time.sleep(0.1)
        _,_,_,f1_val_all_A,f1_val_N,f1_val_A,f1_val_O,_=val_one_epoch(epoch,warm_epoch, total_epoch, model, loss_fn, val_datasloader_A, device,DG_method=DG_method,class_F1=True,recons_model=recons_model,add_recons=add_recons,class_num=class_num) 
        print('f1_val_all_A=0.'+str(f1_val_all_A))
        _,_,_,f1_val_all_B,f1_val_N,f1_val_A,f1_val_O,_=val_one_epoch(epoch,warm_epoch, total_epoch, model, loss_fn, val_datasloader_B, device,DG_method=DG_method,class_F1=True,recons_model=recons_model,add_recons=add_recons,class_num=class_num) 
        print('f1_val_all_B=0.'+str(f1_val_all_B))
        _,_,_,f1_val_all_C,f1_val_N,f1_val_A,f1_val_O,_=val_one_epoch(epoch,warm_epoch, total_epoch, model, loss_fn, val_datasloader_C, device,DG_method=DG_method,class_F1=True,recons_model=recons_model,add_recons=add_recons,class_num=class_num) 
        print('f1_val_all_C=0.'+str(f1_val_all_C))
        _,_,_,f1_val_all_H,f1_val_N,f1_val_A,f1_val_O,_=val_one_epoch(epoch,warm_epoch, total_epoch, model, loss_fn, val_datasloader_H, device,DG_method=DG_method,class_F1=True,recons_model=recons_model,add_recons=add_recons,class_num=class_num) 
        print('f1_val_all_H=0.'+str(f1_val_all_H))
        _,_,_,f1_val_all_L,f1_val_N,f1_val_A,f1_val_O,_=val_one_epoch(epoch,warm_epoch, total_epoch, model, loss_fn, val_datasloader_L, device,DG_method=DG_method,class_F1=True,recons_model=recons_model,add_recons=add_recons,class_num=class_num) 
        print('f1_val_all_L=0.'+str(f1_val_all_L))
        _,_,_,f1_val_all_N,f1_val_N,f1_val_A,f1_val_O,_=val_one_epoch(epoch,warm_epoch, total_epoch, model, loss_fn, val_datasloader_N, device,DG_method=DG_method,class_F1=True,recons_model=recons_model,add_recons=add_recons,class_num=class_num) 
        print('f1_val_all_N=0.'+str(f1_val_all_N))
        
        
        
        

        f1_val_all=int(f1_val_all_A)+int(f1_val_all_B)+int(f1_val_all_C)+int(f1_val_all_H)+int(f1_val_all_L)+int(f1_val_all_N)
        
        print('total_f1_val_all='+str(f1_val_all))
        
        if int(f1_val_all) > best_val_f1:
            best_val_f1 = int(f1_val_all)
            best_model_weights = model.state_dict()  # Save the current model weights
            
            torch.save(best_model_weights, os.path.join(mdst_path, mdst_name))
            


    # Load the best model weights before testing
    model.load_state_dict(torch.load(os.path.join(mdst_path, mdst_name)))

    T_auc_,T_pre_,T_rec_,T_f1_,T_f1_N,T_f1_A,T_f1_O,T_acc_=  \
        test_one_epoch(epoch,warm_epoch, total_epoch, model, loss_fn, test_T_datasloader, device,DG_method=DG_method,class_F1=True,recons_model=recons_model,add_recons=add_recons,class_num=class_num) 
        
    T_auc_dict_list.append(int(T_auc_)  )    
    T_pre_dict_list.append(int(T_pre_)  )
    T_rec_dict_list.append(int(T_rec_)  )
    T_f1_dict_list.append(int(T_f1_)  )
    T_f1_normal_dict_list.append(int(T_f1_N)  )
    T_f1_AF_dict_list.append(int(T_f1_A)  )
    T_f1_other_dict_list.append(int(T_f1_O)  )
    T_acc_dict_list.append(int(T_acc_)  )



    R_auc_,R_pre_,R_rec_,R_f1_,R_f1_N,R_f1_A,R_f1_O,R_acc_=  \
        test_one_epoch(epoch,warm_epoch, total_epoch, model, loss_fn, test_R_datasloader, device,DG_method=DG_method,class_F1=True,recons_model=recons_model,add_recons=add_recons,class_num=class_num) 
        
    R_auc_dict_list.append(int(R_auc_)  )    
    R_pre_dict_list.append(int(R_pre_)  )
    R_rec_dict_list.append(int(R_rec_)  )
    R_f1_dict_list.append(int(R_f1_)  )
    R_f1_normal_dict_list.append(int(R_f1_N)  )
    R_f1_AF_dict_list.append(int(R_f1_A)  )
    R_f1_other_dict_list.append(int(R_f1_O)  )
    R_acc_dict_list.append(int(R_acc_)  )

    
    
T_auc_dict=str(np.mean(T_auc_dict_list))
T_pre_dict=str(np.mean(T_pre_dict_list))
T_rec_dict=str(np.mean(T_rec_dict_list))
T_f1_dict=str(np.mean(T_f1_dict_list))
T_f1_normal_dict=str(np.mean(T_f1_normal_dict_list))
T_f1_AF_dict=str(np.mean(T_f1_AF_dict_list))
T_f1_other_dict=str(np.mean(T_f1_other_dict_list))
T_acc_dict=str(np.mean(T_acc_dict_list))


R_auc_dict=str(np.mean(R_auc_dict_list))
R_pre_dict=str(np.mean(R_pre_dict_list))
R_rec_dict=str(np.mean(R_rec_dict_list))
R_f1_dict=str(np.mean(R_f1_dict_list))
R_f1_normal_dict=str(np.mean(R_f1_normal_dict_list))
R_f1_AF_dict=str(np.mean(R_f1_AF_dict_list))
R_f1_other_dict=str(np.mean(R_f1_other_dict_list))
R_acc_dict=str(np.mean(R_acc_dict_list))

# 写入csv

header=[
        'train\\test\t',
        'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

        # round

results=[
        [
        'T',
        T_auc_dict,T_pre_dict,T_rec_dict,T_f1_dict,T_f1_normal_dict,T_f1_AF_dict,T_f1_other_dict,T_acc_dict+'\t',
         ],
        ]

with open('log/jixian_zhijiaGRL_0308_T.csv','w',encoding='utf-8')as file_obj:
    writer=csv.writer(file_obj)
    writer.writerow(header)
    for result in results:
        writer.writerow(result)
        
        
header=[
        'train\\test\t',
        'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

        # round

results=[
        [
        'R',
        R_auc_dict,R_pre_dict,R_rec_dict,R_f1_dict,R_f1_normal_dict,R_f1_AF_dict,R_f1_other_dict,R_acc_dict+'\t',
         ],
        ]

with open('log/jixian_zhijiaGRL_0308_R.csv','w',encoding='utf-8')as file_obj:
    writer=csv.writer(file_obj)
    writer.writerow(header)
    for result in results:
        writer.writerow(result)

ok
train data C
train data size : 24194
train label class :  [ 6719.  2039. 15436. 24194.     0.     0.     0.     0.     0.]
train data A
train data size : 8274
train label class :  [1393.  480. 6401.    0. 8274.    0.    0.    0.    0.]
train data L
train data size : 16824
train label class :  [13511.  1158.  2155.     0.     0. 16824.     0.     0.     0.]
train data N
train data size : 8516
train label class :  [1462. 1769. 5285.    0.    0.    0. 8516.    0.    0.]
train data B
train data size : 5501
train label class :  [ 742.  859. 3900.    0.    0.    0.    0. 5501.    0.]
train data H
train data size : 20616
train label class :  [11152.   557.  8907.     0.     0.     0.     0.     0. 20616.]
total train data shape =  (83925, 1000, 1)
total train label shape =  (83925, 9)
total train data class =  [34979.  6862. 42084. 24194.  8274. 16824.  8516.  5501. 20616.]
val data A
val data size : 2069
val label class :  [ 359.   90. 1620.]
total val data shape =  (2069, 1000, 1)
total 

epoch 0 loss: 0.6100: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 36.55it/s]


f1_val_all_A=0.629


epoch 0 loss: 0.6047: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 35.57it/s]


f1_val_all_B=0.699


...

epoch 6 loss: 0.4954: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 43.38it/s]


f1_val_all_L=0.724


epoch 6 loss: 0.3096: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 35.41it/s]


f1_val_all_N=0.875
total_f1_val_all=4768
_____________________________________
 


epoch 7 loss: 0.4797:  34%|███████████████████████████████████████████████████████████████████                                                                                                                                     | 220/656 [00:13<00:27, 16.05it/s]

In [6]:
print(1)